reading in 72 stacks of 2048^2 88 slices (72, 88, 2048, 2048) consumed 50 GB of memory and take 22 minutes. 

In [ ]:
import os
import numpy as np 
from pathlib import Path
from tifffile import imread
from tifffile import imsave

file_folder = Path('/Volumes/FlashSSD2/1-460')
files = [item for item in os.listdir(file_folder) if item.endswith('.tif') and 'C00' in item and not item.startswith('.')]
files.sort()

In [ ]:
i0_C00_files = [item for item in files if 'I0' in item]
i1_C00_files = [item for item in files if 'I1' in item]
i0_C00_files.sort()
i0_C00_files.sort()

In [ ]:
sample_image = imread(file_folder / i0_C00_files[0])
slices = sample_image.shape[0]
pixels = sample_image.shape[1]

xy_scale_factor = 8
z_scale_factor = 2
xy_scaled_pixels = int(pixels / xy_scale_factor)
z_scaled_pixels = int(slices / z_scale_factor)

frames = len(i0_C00_files)

if len(i0_C00_files) == len(i1_C00_files):
    i0_C00 = np.zeros((frames, z_scaled_pixels, xy_scaled_pixels, xy_scaled_pixels), dtype = 'uint16')
    i1_C00 = np.zeros((frames, z_scaled_pixels, xy_scaled_pixels, xy_scaled_pixels), dtype = 'uint16')
    c00 = np.zeros((frames, z_scaled_pixels, xy_scaled_pixels, xy_scaled_pixels), dtype = 'uint16')
    print('finished creating empty arrays')
else:
    print('mismatch in the number of left sided and right sided illumination frames')

In [ ]:
for index in range(frames):
    print(f'starting to load time point {index}')
    i0_C00[index] = imread(file_folder / i0_C00_files[index])[::z_scale_factor,::xy_scale_factor,::xy_scale_factor]
    print('finished loading left side illumination for channel 1')
    i1_C00[index] = imread(file_folder / i1_C00_files[index])[::z_scale_factor,::xy_scale_factor,::xy_scale_factor]
    print('finihed loading right side illumination for channel 1')
    c00[index] = np.maximum(i0_C00[index], i1_C00[index])
    print('finished calculating max projection of left and right sided illumination for channel 1')
    print(f'{index/frames*100}% finished downsampling all frames')


In [ ]:
imsave('/Users/bementmbp/Desktop/test.tif', c00)

81.4 / 10 = 8.14 mb / frame

461 frames * 8.14 mb/frame = 3752.54 mb ~3.7GB

In [ ]:
import napari
%gui qt
viewer = napari.Viewer()

In [ ]:
aspect_xz = (2.5 * z_scale_factor) / ((6.4/16)*xy_scale_factor)
viewer.add_image(c00, contrast_limits=(0,2500), colormap='gray', blending='additive', scale=(aspect_xz, 1,1))

In [ ]:
i0_C00 = np.zeros((len(i0_C00_files), 179, 512, 512), dtype = 'uint16')
i1_C00 = np.zeros((len(i1_C00_files), 179, 512, 512), dtype = 'uint16')
i0_C02 = np.zeros((len(i0_C02_files), 179, 512, 512), dtype = 'uint16')
i1_C02 = np.zeros((len(i1_C02_files), 179, 512, 512), dtype = 'uint16')
c00 = np.zeros((len(i1_C02_files), 179, 512, 512), dtype = 'uint16')
c02 = np.zeros((len(i1_C02_files), 179, 512, 512), dtype = 'uint16')

for index, image in enumerate(i0_C00_files):
    i0_C00[index] = imread(file_folder / i0_C00_files[index])[::,::4,::4]
    i1_C00[index] = imread(file_folder / i1_C00_files[index])[::,::4,::4]
    i0_C02[index] = imread(file_folder / i0_C02_files[index])[::,::4,::4]
    i1_C02[index] = imread(file_folder / i1_C02_files[index])[::,::4,::4]
    c00[index] = np.maximum(i0_C00[index], i1_C00[index])
    c02[index] = np.maximum(i0_C02[index], i1_C02[index])
    print(f'{round(index/len(i0_C00_files)*100,2)}% finished reading')


In [ ]:
test = imread('/Volumes/Extreme_Pro/20220318_180146_Itsn1HighE1Series/S000_t000000_V000_R0000_X000_Y000_C00_I1_D0_P00179.tif')
viewer.add_image(test)

In [ ]:
aspect_xz = 2.5 / 0.4
viewer.add_image(test_im, scale=(aspect_xz, 1,1))

In [ ]:
#from dask_image.imread import imread
import dask.array.image
from pathlib import Path
base = Path.cwd()
images = dask.array.image.imread('/Volumes/Extreme Pro/20220316_125052_CntrlEmb/0_right_side_only/*.tif') # Dask array type, with shape (200, 520, 696) (images, ydim, xdim)

In [ ]:
images

In [ ]:

print(test.shape)

In [ ]:
import napari
%gui qt
viewer = napari.Viewer()

In [ ]:
viewer.add_image(images)

Pre-process the data (here we will just smooth):

In [ ]:
from dask_image import ndfilters
smoothed = ndfilters.gaussian_filter(images, sigma=[0, 1, 1])
# smoothed is a Dask array of the smoothed images

Let's try an absolute threshold, and display the results in Napari:

In [ ]:
import napari
%gui qt
viewer = napari.Viewer()
absolute_threshold = smoothed > 160
viewer.add_image(absolute_threshold, opacity = 0.5)
viewer.add_image(images, contrast_limits=[0, 2000], blending='additive')

This works okay, but not great, and NOT equally for all images

In [ ]:
from napari.utils import nbscreenshot
nbscreenshot(viewer)

Instead, let's try a local threshold instead (https://image.dask.org/en/2021.12.0/dask_image.ndfilters.html?highlight=threshold_local#dask_image.ndfilters.threshold_local)

In [ ]:
thresh = ndfilters.threshold_local(image = smoothed, block_size = images.chunksize)
threshold_images = smoothed > thresh                            # images.chunksize is (1, 520, 696), one image 

Let's get rid of our old layers

In [ ]:
def removeLayers():
    layers = viewer.layers
    while len(layers) > 0:
        layers.remove(layers[0])    

removeLayers()

And visualize our new threshold. Much better!

In [ ]:
viewer.add_image(threshold_images, opacity = 0.5)
viewer.add_image(images, contrast_limits=[0, 2000], blending='additive')

In [ ]:
from napari.utils import nbscreenshot
nbscreenshot(viewer)

Let's clean up our threshold by running a binary "opening" (erosion followed by dilation) http://image.dask.org/en/latest/dask_image.ndmorph.html#dask_image.ndmorph.binary_opening

NOTE: Dask apparently does not support watershedding...

In [ ]:
from dask_image import ndmorph
import numpy as np

structuring_element = np.array([        # this is equivalent to the defaults 2D structuring element, so we don't influence neighboring slices in our array
    [[0, 0, 0], [0, 0, 0], [0, 0, 0]],  # [[0, 1, 0],
    [[0, 1, 0], [1, 1, 1], [0, 1, 0]],  #  [1, 1, 1],
    [[0, 0, 0], [0, 0, 0], [0, 0, 0]]]) #  [0, 1, 0]]

binary_images = ndmorph.binary_opening(threshold_images, structure=structuring_element)

In [ ]:
viewer.add_image(binary_images)

Let's create labels for the binary features so we can analyze them 

In [ ]:
from dask_image import ndmeasure

# Create labelled mask
label_images, num_features = ndmeasure.label(binary_images, structuring_element)
index = np.arange(num_features - 1) + 1 
viewer.add_labels(label_images)

Now each feature is labeled and can be analyzed independently

In [ ]:
from napari.utils import nbscreenshot
nbscreenshot(viewer)

Let's count how many nuclei we have thresholded

In [ ]:
print("Number of nuclei:", num_features.compute())

We can also make measurements like area and mean intensity

In [ ]:
area = ndmeasure.area(images, label_images, index)
mean_intensity = ndmeasure.mean(images, label_images, index)

Which we can plot interactively

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(area, mean_intensity, alpha=0.5)
plt.gca().update(dict(title="Area vs mean intensity", xlabel='Area (pixels)', ylabel='Mean intensity'))
plt.show()